In [1]:
import pandas as pd
import numpy as np
import Stock_Func

In [2]:
train_data = pd.read_csv('train.csv', sep=',', index_col=0)
test_data = pd.read_csv('test.csv', sep=',', index_col=0)

## Add some attributes as we do in Logistic Regression part

In [3]:
test_data = Stock_Func.NewFeature(test_data)
train_data = Stock_Func.NewFeature(train_data)
test_data = Stock_Func.Up_Down(test_data)
train_data = Stock_Func.Up_Down(train_data)
test_data = Stock_Func.AddRSI6(test_data)
train_data = Stock_Func.AddRSI6(train_data)
test_data = Stock_Func.AddRSI12(test_data)
train_data = Stock_Func.AddRSI12(train_data)
train_data = train_data.drop(train_data.index[[0,1,2,3,4,5,6,7,
                                               8,9,10,11,
                                               train_data.shape[0]-1]])
test_data = test_data.drop(test_data.index[[0,1,2,3,4,5,6,7,
                                            8,9,10,11,
                                            test_data.shape[0]-1]])
train_data = Stock_Func.KD_variation(train_data)
test_data = Stock_Func.KD_variation(test_data)
train_data = Stock_Func.KD_strength(train_data)
test_data = Stock_Func.KD_strength(test_data)
train_data = Stock_Func.RSI_variation(train_data)
test_data = Stock_Func.RSI_variation(test_data)

# Train the model
## Use the original data to predict the up and down.
#### Attributes:	"Open Price"	"Close Price"	"Low Price"	  "High Price"

In [4]:
drop_list = ['Volume','RSV','Up_Down','RSI6','RSI12','RSI_variation',
            'Kvalue','Dvalue','KD_variation','KD_strength']
_train_data = train_data
_test_data = test_data
_train_data = train_data.drop(drop_list, axis = 1)
_test_data = test_data.drop(drop_list, axis = 1) 

x_train = _train_data.loc[:, _train_data.columns != 'RisePredict']
y_train = _train_data['RisePredict']
x_test = _test_data.loc[:, _test_data.columns != 'RisePredict']
y_test = _test_data['RisePredict']

In [5]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [6]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

Using TensorFlow backend.


In [7]:
classifier = Sequential()
classifier.add(Dense(units = 128, kernel_initializer = 'uniform', activation = 'relu', 
                     input_dim = x_train.shape[1]))
classifier.add(Dense(units = 128, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
classifier.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy'])
classifier.fit(x_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
2251/2251 [==============================] - 0s 170us/step - loss: 0.2487 - acc: 0.5429
Epoch 2/100
2251/2251 [==============================] - 0s 112us/step - loss: 0.2484 - acc: 0.5464
Epoch 3/100
2251/2251 [==============================] - 0s 116us/step - loss: 0.2481 - acc: 0.5464
Epoch 4/100
2251/2251 [==============================] - 0s 110us/step - loss: 0.2482 - acc: 0.5464
Epoch 5/100
2251/2251 [==============================] - 0s 122us/step - loss: 0.2481 - acc: 0.5464
Epoch 6/100
2251/2251 [==============================] - 0s 115us/step - loss: 0.2480 - acc: 0.5464
Epoch 7/100
2251/2251 [==============================] - 0s 114us/step - loss: 0.2481 - acc: 0.5464
Epoch 8/100
2251/2251 [==============================] - 0s 110us/step - loss: 0.2479 - acc: 0.5464
Epoch 9/100
2251/2251 [==============================] - 0s 111us/step - loss: 0.2480 - acc: 0.5464
Epoch 10/100
2251/2251 [==============================] - 0s 107us/step - loss: 0.2481 - acc: 0.5464

2251/2251 [==============================] - 0s 118us/step - loss: 0.2476 - acc: 0.5464
Epoch 83/100
2251/2251 [==============================] - 0s 123us/step - loss: 0.2476 - acc: 0.5464
Epoch 84/100
2251/2251 [==============================] - 0s 119us/step - loss: 0.2476 - acc: 0.5464
Epoch 85/100
2251/2251 [==============================] - 0s 127us/step - loss: 0.2476 - acc: 0.5464
Epoch 86/100
2251/2251 [==============================] - 0s 113us/step - loss: 0.2477 - acc: 0.5464
Epoch 87/100
2251/2251 [==============================] - 0s 113us/step - loss: 0.2476 - acc: 0.5464
Epoch 88/100
2251/2251 [==============================] - 0s 121us/step - loss: 0.2475 - acc: 0.5464
Epoch 89/100
2251/2251 [==============================] - 0s 131us/step - loss: 0.2476 - acc: 0.5464
Epoch 90/100
2251/2251 [==============================] - 0s 114us/step - loss: 0.2477 - acc: 0.5464
Epoch 91/100
2251/2251 [==============================] - 0s 118us/step - loss: 0.2477 - acc: 0.5464
Epo

In [8]:
y_pred = classifier.predict(x_test)
y_pred = (y_pred > 0.5)
print(classification_report(y_test,y_pred ))
acc_rate = accuracy_score(y_test.values, y_pred ) * 100
print("Accuracy = %.2f %%" % acc_rate)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       117
           1       0.51      1.00      0.68       122

   micro avg       0.51      0.51      0.51       239
   macro avg       0.26      0.50      0.34       239
weighted avg       0.26      0.51      0.35       239

Accuracy = 51.05 %


/home/cmw/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Use the K/D value to predict the up and down.
#### Attributes:	"High Price"	"Kvalue"	"Dvalue"	"KD_variation"	"KD_strength"

In [9]:
drop_list = ['Open Price','Close Price','Low Price','Volume','RSV','Up_Down','RSI6','RSI12','RSI_variation']
_train_data = train_data
_test_data = test_data
_train_data = train_data.drop(drop_list, axis = 1)
_test_data = test_data.drop(drop_list, axis = 1) 

x_train = _train_data.loc[:, _train_data.columns != 'RisePredict']
y_train = _train_data['RisePredict']
x_test = _test_data.loc[:, _test_data.columns != 'RisePredict']
y_test = _test_data['RisePredict']

In [10]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

/home/cmw/.local/lib/python3.5/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/cmw/.local/lib/python3.5/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/cmw/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
classifier2 = Sequential()
classifier2.add(Dense(units = 128, kernel_initializer = 'uniform', activation = 'relu', 
                     input_dim = x_train.shape[1]))
classifier2.add(Dense(units = 128, kernel_initializer = 'uniform', activation = 'relu'))
classifier2.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
classifier2.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy'])
classifier2.fit(x_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
2251/2251 [==============================] - 0s 183us/step - loss: 0.2486 - acc: 0.5446
Epoch 2/100
2251/2251 [==============================] - 0s 112us/step - loss: 0.2477 - acc: 0.5464
Epoch 3/100
2251/2251 [==============================] - 0s 142us/step - loss: 0.2475 - acc: 0.5464
Epoch 4/100
2251/2251 [==============================] - 0s 143us/step - loss: 0.2471 - acc: 0.5464
Epoch 5/100
2251/2251 [==============================] - 0s 119us/step - loss: 0.2474 - acc: 0.5464
Epoch 6/100
2251/2251 [==============================] - 0s 114us/step - loss: 0.2472 - acc: 0.5464
Epoch 7/100
2251/2251 [==============================] - 0s 113us/step - loss: 0.2469 - acc: 0.5464
Epoch 8/100
2251/2251 [==============================] - 0s 150us/step - loss: 0.2469 - acc: 0.5464
Epoch 9/100
2251/2251 [==============================] - 0s 116us/step - loss: 0.2466 - acc: 0.5464
Epoch 10/100
2251/2251 [==============================] - 0s 124us/step - loss: 0.2473 - acc: 0.5464

2251/2251 [==============================] - 0s 109us/step - loss: 0.2415 - acc: 0.5709
Epoch 83/100
2251/2251 [==============================] - 0s 122us/step - loss: 0.2411 - acc: 0.5686
Epoch 84/100
2251/2251 [==============================] - 0s 111us/step - loss: 0.2410 - acc: 0.5700
Epoch 85/100
2251/2251 [==============================] - 0s 113us/step - loss: 0.2410 - acc: 0.5731
Epoch 86/100
2251/2251 [==============================] - 0s 112us/step - loss: 0.2411 - acc: 0.5677
Epoch 87/100
2251/2251 [==============================] - 0s 107us/step - loss: 0.2409 - acc: 0.5646
Epoch 88/100
2251/2251 [==============================] - 0s 110us/step - loss: 0.2407 - acc: 0.5686
Epoch 89/100
2251/2251 [==============================] - 0s 107us/step - loss: 0.2411 - acc: 0.5629
Epoch 90/100
2251/2251 [==============================] - 0s 108us/step - loss: 0.2402 - acc: 0.5677
Epoch 91/100
2251/2251 [==============================] - 0s 121us/step - loss: 0.2405 - acc: 0.5709
Epo

In [12]:
y_pred = classifier2.predict(x_test)
y_pred = (y_pred > 0.5)
print(classification_report(y_test,y_pred ))
acc_rate = accuracy_score(y_test.values, y_pred ) * 100
print("Accuracy = %.2f %%" % acc_rate)

              precision    recall  f1-score   support

           0       0.61      0.09      0.16       117
           1       0.52      0.94      0.67       122

   micro avg       0.53      0.53      0.53       239
   macro avg       0.57      0.52      0.42       239
weighted avg       0.56      0.53      0.42       239

Accuracy = 52.72 %


## Use the RSI6/RSI12 value to predict the up and down.
#### Attributes:	"High Price"	"RSI12"		"RSI_variation"

In [13]:
drop_list = ['Open Price','Close Price','Low Price','Volume','RSV',
             'RSI6','Up_Down','Kvalue','Dvalue','KD_variation','KD_strength']
_train_data = train_data
_test_data = test_data
_train_data = train_data.drop(drop_list, axis = 1)
_test_data = test_data.drop(drop_list, axis = 1) 

x_train = _train_data.loc[:, _train_data.columns != 'RisePredict']
y_train = _train_data['RisePredict']
x_test = _test_data.loc[:, _test_data.columns != 'RisePredict']
y_test = _test_data['RisePredict']

In [14]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

/home/cmw/.local/lib/python3.5/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/cmw/.local/lib/python3.5/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/cmw/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
classifier3 = Sequential()
classifier3.add(Dense(units = 128, kernel_initializer = 'uniform', activation = 'relu', 
                     input_dim = x_train.shape[1]))
classifier3.add(Dense(units = 128, kernel_initializer = 'uniform', activation = 'relu'))
classifier3.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
classifier3.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy'])
classifier3.fit(x_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
2251/2251 [==============================] - 0s 179us/step - loss: 0.2488 - acc: 0.5433
Epoch 2/100
2251/2251 [==============================] - 0s 127us/step - loss: 0.2484 - acc: 0.5464
Epoch 3/100
2251/2251 [==============================] - 0s 121us/step - loss: 0.2480 - acc: 0.5464
Epoch 4/100
2251/2251 [==============================] - 0s 117us/step - loss: 0.2474 - acc: 0.5464
Epoch 5/100
2251/2251 [==============================] - 0s 121us/step - loss: 0.2483 - acc: 0.5464
Epoch 6/100
2251/2251 [==============================] - 0s 118us/step - loss: 0.2478 - acc: 0.5473
Epoch 7/100
2251/2251 [==============================] - 0s 116us/step - loss: 0.2476 - acc: 0.5478
Epoch 8/100
2251/2251 [==============================] - 0s 121us/step - loss: 0.2474 - acc: 0.5464
Epoch 9/100
2251/2251 [==============================] - 0s 118us/step - loss: 0.2476 - acc: 0.5540
Epoch 10/100
2251/2251 [==============================] - 0s 121us/step - loss: 0.2475 - acc: 0.5491

2251/2251 [==============================] - 0s 139us/step - loss: 0.2429 - acc: 0.5637
Epoch 83/100
2251/2251 [==============================] - 0s 110us/step - loss: 0.2428 - acc: 0.5629
Epoch 84/100
2251/2251 [==============================] - 0s 132us/step - loss: 0.2427 - acc: 0.5700
Epoch 85/100
2251/2251 [==============================] - 0s 122us/step - loss: 0.2426 - acc: 0.5695
Epoch 86/100
2251/2251 [==============================] - 0s 122us/step - loss: 0.2425 - acc: 0.5695
Epoch 87/100
2251/2251 [==============================] - 0s 162us/step - loss: 0.2425 - acc: 0.5691
Epoch 88/100
2251/2251 [==============================] - 0s 146us/step - loss: 0.2427 - acc: 0.5677
Epoch 89/100
2251/2251 [==============================] - 0s 130us/step - loss: 0.2424 - acc: 0.5726
Epoch 90/100
2251/2251 [==============================] - 0s 108us/step - loss: 0.2423 - acc: 0.5735
Epoch 91/100
2251/2251 [==============================] - 0s 130us/step - loss: 0.2421 - acc: 0.5700
Epo

In [16]:
y_pred = classifier3.predict(x_test)
y_pred = (y_pred > 0.5)
print(classification_report(y_test,y_pred ))
acc_rate = accuracy_score(y_test.values, y_pred ) * 100
print("Accuracy = %.2f %%" % acc_rate)

              precision    recall  f1-score   support

           0       0.43      0.03      0.05       117
           1       0.51      0.97      0.67       122

   micro avg       0.51      0.51      0.51       239
   macro avg       0.47      0.50      0.36       239
weighted avg       0.47      0.51      0.36       239

Accuracy = 50.63 %
